In [9]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import xlsxwriter
import datetime

In [10]:
# ususaarios de base de datos local

#usuario = "care_consulta"
#contrasena = "CareAssistance!10"
#url_servidor = "10.0.10.22"
# puerto = '3306'
# esquema = 'EspoCRM2'
# plugin_autenticacion = 'mysql_native_password'

# usuarios de base de datos Simtastic ##########################################

usuario = 'siempreread'
contrasena = 'S13mpr3_Ch1l3'
url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "EspoCRM2"
plugin_autenticacion = "mysql_native_password"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

In [11]:
sql = "select cas.number as 'Id',trim(con.rut) as 'Rut Paciente',trim(con.rut_asegurado) as 'Rut Titular',con.relacion_con_asegurado as 'Relación con asegurado', cas.area as 'Prestacion',"
sql2 = " cas.created_at as 'Fecha Gestión', con.empleador AS 'Empleador', cas.channel as 'Canal', trim(con.campana_paciente) as 'Campaña', con.holding as 'holding'"
sql3 = "from EspoCRM2.case cas left join EspoCRM2.contact con on cas.contact_id = con.id where con.campana_paciente like '%zurich%' AND cas.deleted = 0 and cas.status <> 'Cancelled' order by cas.number asc;"

# archivo de salida modificar segun archivo
archivo = "C:/Users/jmartinez/Desktop/Informes/Metlife/zurich.xlsx"

fin_sql = sql + sql2 + sql3

In [12]:
conchaytoro = pd.read_sql_query(fin_sql, motor_mysql_mariadb)



conchaytoro.head(10)


,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña,holding
0,251744,15809832-6,15879832-6,AS,Siempre - Medicina,2022-10-07 15:50:50,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA
1,251897,15809832-6,15879832-6,AS,Siempre - Front,2022-10-07 19:01:29,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA
2,251913,15809832-6,15879832-6,AS,Siempre - Front,2022-10-07 19:20:22,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA
3,253960,15719080-6,15719080-6,AS,Siempre - Deportiva,2022-10-13 18:13:18,ANTOFAGASTA MINERALS S.A.,Sin canal,Zurich,AMSA
4,253963,15809832-6,15879832-6,AS,Siempre - Deportiva,2022-10-13 18:14:08,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA
5,254010,19526017-6,19526017-6,AS,Siempre - Deportiva,2022-10-13 18:51:50,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA
6,254015,17375597-k,17375597-k,AS,Siempre - Deportiva,2022-10-13 18:55:06,ANTOFAGASTA MINERALS S.A.,Sin canal,Zurich,AMSA
7,254021,23296905-9,23296905-9,AS,Siempre - Deportiva,2022-10-13 18:57:53,ANTOFAGASTA MINERALS S.A.,Sin canal,Zurich,AMSA
8,254027,26563920-8,26563920-8,AS,Siempre - Deportiva,2022-10-13 19:04:32,ANTOFAGASTA MINERALS S.A.,Sin canal,Zurich,AMSA
9,254362,19526017-6,19526017-6,AS,Siempre - Deportiva,2022-10-14 14:42:29,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA


In [13]:
conchaytoro.shape

(39, 10)

In [14]:
relacion = {
    "HI": "Hijo/a",
    "AS": "Titular",
    "CO": "Cónyuge",
    "P": "Padre",
    "O": "Otros",
    "M": "Madre",
    "H": "Hijo/a",
    "NI":"Nieto/a"
}
conchaytoro.replace({"Relación con asegurado": relacion}, inplace=True)

prestaciones = {
    "Deporte": "Programa Deportivo",
    "Nutrición": "Programa Nutricional",
    #"Pcte. Crónico": "Programa Enfermería",
    "Orientación": "Orientación Hospitalaria",
    "Psicología": "Programa de Psicología",
    "Psicología": "Programa de Psicología",
    "Medicina": "Doctor Online",
    "Deporte": "Programa Deportivo",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Pcte. Crónico": "Programa de Hospitalización Domiciliaria",
    "Medicina": "Doctor Online",
    "Psicología": "Programa de Psicología",
    "Nutrición": "Programa Nutricional",
    "Deporte": "Programa Deportivo",
    "Fidelización": "Fidelización",
    "Orientación": "Orientación Hospitalaria",
    #"Pcte. Crónico": "Programa Enfermería",
    "Medicina": "Doctor Online",
    "Agenda": "Orientación Hospitalaria",
    "Fidelización y SAC": "Orientación Hospitalaria",
    "Ingreso dep.  seguro": "Programa Deportivo",
    #"Clínica del Sueño": "Programa Enfermería",
    "Ingreso dep. seguro": "Programa Deportivo",
    "APSSex": "Programa de Psicología",
    "Kinesiología HC": "Programa Deportivo",
    "Siempre - Medicina": "Doctor Online",
    "Siempre - Orientación": "Orientación Hospitalaria",
    "Siempre - Agenda": "Orientación Hospitalaria",
    "Siempre - Deportiva": "Programa Deportivo",
    "Siempre - Psicología": "Programa de Psicología",
    "Siempre - Nutrición": "Programa Nutricional",
    "Siempre  - Front": "Orientación Hospitalaria",
    "Siempre - Clínica del Sueño": "Clinica del Sueño",
    #"Siempre - Enfermería": "Programa Enfermería",
    "Telemedicina": "Doctor Online",
    "Nutricion": "Programa Nutricional",
    "Front": "Orientación Hospitalaria",
    "Deportivo": "Programa Deportivo",
}
#conchaytoro.replace({"Prestacion": prestaciones}, inplace=True)


conchaytoro["derty"] = pd.to_datetime(
    conchaytoro["Fecha Gestión"], format="dd-md-yyyy h:mm"
)
conchaytoro.derty = conchaytoro.derty.dt.strftime("%d-%m-%Y %H:%M")
# conchaytoro['Fecha Gestión'] = conchaytoro['Fecha Gestión'].apply(lambda _: datetime.strptime(_,"%d-%m-%Y, %H:%M"))

In [15]:
conchaytoro.head(50)

,Id,Rut Paciente,Rut Titular,Relación con asegurado,Prestacion,Fecha Gestión,Empleador,Canal,Campaña,holding,derty
0,251744,15809832-6,15879832-6,Titular,Siempre - Medicina,2022-10-07 15:50:50,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,07-10-2022 15:50
1,251897,15809832-6,15879832-6,Titular,Siempre - Front,2022-10-07 19:01:29,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,07-10-2022 19:01
2,251913,15809832-6,15879832-6,Titular,Siempre - Front,2022-10-07 19:20:22,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,07-10-2022 19:20
3,253960,15719080-6,15719080-6,Titular,Siempre - Deportiva,2022-10-13 18:13:18,ANTOFAGASTA MINERALS S.A.,Sin canal,Zurich,AMSA,13-10-2022 18:13
4,253963,15809832-6,15879832-6,Titular,Siempre - Deportiva,2022-10-13 18:14:08,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,13-10-2022 18:14
...,...,...,...,...,...,...,...,...,...,...,...
34,265346,19526017-6,19526017-6,Titular,Siempre - Deportiva,2022-11-10 17:04:18,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,10-11-2022 17:04
35,266760,24678750-6,24678750-6,Titular,Siempre - Deportiva,2022-11-14 18:06:19,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,14-11-2022 18:06
36,266833,16128822-5,24678750-6,Cónyuge,Siempre - Medicina,2022-11-14 19:34:32,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,14-11-2022 19:34
37,268773,16128822-5,24678750-6,Cónyuge,Siempre - Medicina,2022-11-18 20:38:47,ANTOFAGASTA MINERALS S.A.,Llamada telefónica,Zurich,AMSA,18-11-2022 20:38


In [16]:
conchaytoro.to_excel(archivo, index=False)